In [ ]:
!pip install Levenshtein

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.4/177.4 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 8.8 MB/s eta 0:00:00


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import Levenshtein

In [ ]:
# Load your text dataset from a text file
def load_dataset(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        dataset = file.readlines()
    return dataset

In [ ]:
# Load your dataset
file_path = '/content/reddit50.txt'
dataset = load_dataset(file_path)

In [ ]:
# # Sample function to generate responses to questions
# def generate_response(question, model, tokenizer, max_length=50):
#     input_sequence = tokenizer.texts_to_sequences([question])
#     padded_sequence = pad_sequences(input_sequence, maxlen=max_length, padding='post')
#     predicted_sequence = model.predict(padded_sequence)
#     response = tokenizer.sequences_to_texts(predicted_sequence)[0]
#     return response
import numpy as np
#from tensorflow.keras.preprocessing.sequence import pad_sequences

# Function to generate response using the model
def generate_response(question, model, tokenizer, start_token, end_token, max_encoder_sequence_length, max_decoder_sequence_length):
    # Tokenize the question
    question_tokens = tokenizer.texts_to_sequences([question])

    # Pad the question sequence
    question_padded = pad_sequences(question_tokens, maxlen=max_encoder_sequence_length, padding='post')

    # Initialize the decoder input sequence with the start token
    decoder_input_sequence = np.array([[start_token]])

    # Initialize the decoder output sequence
    decoder_output_sequence = None

    # Iterate until the maximum decoder sequence length is reached or the end token is generated
    for _ in range(max_decoder_sequence_length):
        # Predict the next token in the sequence
        decoder_output = model.predict([question_padded, decoder_input_sequence])

        # Get the predicted token (argmax of the softmax output)
        predicted_token = np.argmax(decoder_output)

        # Append the predicted token to the decoder output sequence
        if decoder_output_sequence is None:
            decoder_output_sequence = np.array([[predicted_token]])
        else:
            decoder_output_sequence = np.append(decoder_output_sequence, [[predicted_token]], axis=1)

        # If the end token is predicted, stop the iteration
        if np.any(predicted_token == end_token):
            break

        # Update the decoder input sequence with the predicted token
        decoder_input_sequence = np.array([[predicted_token]])

    # Decode the generated output sequence
    generated_answer = tokenizer.sequences_to_texts(decoder_output_sequence)[0]

    return generated_answer


In [ ]:
# Define a function to calculate string similarity using Levenshtein distance
def string_similarity(str1, str2):
    return 1 - (Levenshtein.distance(str1.lower(), str2.lower()) / max(len(str1), len(str2)))


In [ ]:
questions_answers = [
{"question": "What major event occurred in late 2022 involving a tech company?", "answer": "A tech company bought another company and worked on merging systems and products, completing integration earlier this month."},
{"question": "What was the expectation after the integration of the two companies?", "answer": "The expectation was for full integration of headquarters teams of the acquired company into the parent company starting in March."},
{"question": "What happened to senior management, including the former CEO, after the integration?", "answer": "Senior management, including the former CEO, recently moved positions within the new company."},
{"question": "What was the unexpected news delivered via email from the CEO?", "answer": "The email stated that due to the current business climate, difficult decisions had been made, leading to layoffs."},
{"question": "How did the layoffs affect the entire team?", "answer": "The layoffs affected everyone, including leaders, leaving them feeling blindsided and questioning their future."},
{"question": "How long has the person been searching for a new job?", "answer": "The person has been searching for a new job for seven months."},
{"question": "What significant event happened in the tech industry that the person read about on CNN?", "answer": "There was an AT&T outage likely caused by a hacker attack, reflecting potential issues in big tech corporations."},
{"question": "What concerns does the person have about certain programs in the tech industry?", "answer": "The person is concerned about programs that bring in H1B workers for jobs that could be filled by subject matter experts, potentially hurting American workers."},
{"question": "How does the person plan to cope with the stress of unemployment?", "answer": "The person plans to simplify life, possibly by moving to a cheaper area and finding a stable government job."},
{"question": "What lesson did the person learn from their career in software quality assurance?", "answer": "The person learned to conduct their career like a business, focusing on personal growth and marketability."},
{"question": "How did the layoffs affect the person financially and emotionally?", "answer": "The person experienced financial strain due to being the sole income provider for their family, leading to stress and anxiety."},
{"question": "What advice did the person receive after being laid off?", "answer": "The person received advice to continue networking, learn new skills, and consider contract work while searching for a new job."},
{"question": "How did the layoffs affect the person's perspective on loyalty to a company?", "answer": "The person shifted from a perspective of company loyalty to prioritizing personal growth and marketability."},
{"question": "What advice did the person offer to others facing layoffs?", "answer": "The person advised others to stay proactive, continue learning, and network to improve their chances of finding a new job."},
{"question": "What was the person's experience after being laid off from their job in tech?", "answer": "The person faced challenges in finding a new job despite their experience and qualifications."},
{"question": "How did the layoff affect the person's financial situation?", "answer": "The layoff significantly reduced the person's income, leading to financial stress and uncertainty."},
{"question": "What were some strategies the person used to survive financially after the layoff?", "answer": "The person utilized state programs for financial assistance, started a small business, and sought mortgage assistance."},
{"question": "How did the layoff impact the person's mental health?", "answer": "The layoff took a toll on the person's mental health, leading to feelings of hopelessness and anxiety."},
{"question": "What advice did the person offer to others facing layoffs?", "answer": "The person advised others to take action quickly, seek support, and explore alternative sources of income."},
{"question": "How did the person's perspective on job security change after being laid off?", "answer": "The person no longer viewed job security as guaranteed and emphasized the importance of being prepared for unexpected layoffs."}
]

In [ ]:
# Preprocess dataset and prepare tokenizer
questions = [qa["question"] for qa in questions_answers]
answers = [qa["answer"] for qa in questions_answers]


In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(questions + answers)
vocab_size = len(tokenizer.word_index) + 1

In [ ]:
# Convert text data to tokenized sequences
questions_sequences = tokenizer.texts_to_sequences(questions)
answers_sequences = tokenizer.texts_to_sequences(answers)

In [ ]:
max_question_length = max(len(seq) for seq in questions_sequences)
max_answer_length = max(len(seq) for seq in answers_sequences)



In [ ]:
# Pad sequences
questions_padded = pad_sequences(questions_sequences, maxlen=max_question_length, padding='post')
answers_padded = pad_sequences(answers_sequences, maxlen=max_answer_length, padding='post')


In [ ]:
# Define your model architecture
embedding_dim = 100
hidden_units = 256

In [ ]:

def build_encoder(vocab_size, embedding_dim, max_question_length, hidden_units):
    encoder_inputs = tf.keras.layers.Input(shape=(max_question_length,))
    encoder_embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)(encoder_inputs)
    encoder_lstm = tf.keras.layers.LSTM(hidden_units, return_state=True)
    encoder_outputs, state_h, state_c = encoder_lstm(encoder_embedding)
    encoder_states = [state_h, state_c]

    return encoder_inputs, encoder_outputs, encoder_states

# Example usage:
encoder_inputs, encoder_outputs, encoder_states = build_encoder(vocab_size, embedding_dim, max_question_length, hidden_units)



In [ ]:


def build_decoder(vocab_size, embedding_dim, max_answer_length, hidden_units, encoder_states):
    decoder_inputs = tf.keras.layers.Input(shape=(max_answer_length,))
    decoder_embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)(decoder_inputs)
    decoder_lstm = tf.keras.layers.LSTM(hidden_units, return_sequences=True, return_state=True)
    decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
    decoder_dense = tf.keras.layers.Dense(vocab_size, activation='softmax')
    decoder_outputs = decoder_dense(decoder_outputs)

    return decoder_inputs, decoder_outputs

# Example usage:
decoder_inputs, decoder_outputs = build_decoder(vocab_size, embedding_dim, max_answer_length, hidden_units, encoder_states)



In [ ]:
def create_model(vocab_size, embedding_dim, hidden_units, max_question_length, max_answer_length):
    # Define encoder inputs
    encoder_inputs = tf.keras.layers.Input(shape=(max_question_length,))
    encoder_embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)(encoder_inputs)
    encoder_lstm = tf.keras.layers.LSTM(hidden_units, return_state=True)
    encoder_outputs, state_h, state_c = encoder_lstm(encoder_embedding)
    encoder_states = [state_h, state_c]

    # Define decoder inputs
    decoder_inputs = tf.keras.layers.Input(shape=(max_answer_length,))
    decoder_embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)(decoder_inputs)
    decoder_lstm = tf.keras.layers.LSTM(hidden_units, return_sequences=True, return_state=True)
    decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
    decoder_dense = tf.keras.layers.Dense(vocab_size, activation='softmax')
    decoder_outputs = decoder_dense(decoder_outputs)

    # Define the model
    model = tf.keras.models.Model([encoder_inputs, decoder_inputs], decoder_outputs)

    return model


In [ ]:
# model = create_model(vocab_size, embedding_dim, hidden_units, max_question_length, max_answer_length)
model = create_model(vocab_size, embedding_dim, hidden_units, max_question_length, max_answer_length)


In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')


In [ ]:
# Train your model
model.fit([questions_padded, answers_padded], answers_padded, epochs=6, verbose=1)

Epoch 1/6
1/1 [==============================] - 0s 40ms/step - loss: 5.2058
Epoch 2/6
1/1 [==============================] - 0s 27ms/step - loss: 4.9093
Epoch 3/6
1/1 [==============================] - 0s 25ms/step - loss: 4.2589
Epoch 4/6
1/1 [==============================] - 0s 29ms/step - loss: 3.9144
Epoch 5/6
1/1 [==============================] - 0s 28ms/step - loss: 4.1847
Epoch 6/6
1/1 [==============================] - 0s 18ms/step - loss: 4.1709


In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score
import matplotlib.pyplot as plt

# Function to evaluate the model
def evaluate_model(ground_truth, threshold):
    y_true = []
    y_pred = []

    for data_point in questions_answers:
        question = data_point["question"]
        correct_answer = data_point["answer"]
        print(question)
        print(correct_answer)


        max_encoder_sequence_length = 15
        start_token = 1
        max_decoder_sequence_length = 15
        end_token = 2
        # Generate predicted answer using the model
        predicted_answer = generate_response(question, model, tokenizer, start_token, end_token, max_encoder_sequence_length, max_decoder_sequence_length)
        #print(predicted_answer)
        # Compute similarity score between predicted and correct answers
        similarity_score = string_similarity(predicted_answer, correct_answer)

        # Determine if prediction is correct based on threshold
        if similarity_score >= threshold:
            y_pred.append(1)
        else:
            y_pred.append(0)

        # Add true label
        y_true.append(1)  # Assuming all ground truth answers are correct

    # Calculate precision, recall, and F1 score
    precision = precision_score(y_true, y_pred, pos_label=1)
    recall = recall_score(y_true, y_pred, pos_label=1)
    f1 = f1_score(y_true, y_pred, pos_label=1)

    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1 Score: {f1}")


# Evaluate the model
threshold = 0.25  # Adjust threshold based on your requirements
evaluate_model(questions_answers, threshold)


What major event occurred in late 2022 involving a tech company?
A tech company bought another company and worked on merging systems and products, completing integration earlier this month.
1/1 [==============================] - 0s 36ms/step
What was the expectation after the integration of the two companies?
The expectation was for full integration of headquarters teams of the acquired company into the parent company starting in March.
1/1 [==============================] - 0s 29ms/step
What happened to senior management, including the former CEO, after the integration?
Senior management, including the former CEO, recently moved positions within the new company.
1/1 [==============================] - 0s 35ms/step
What was the unexpected news delivered via email from the CEO?
The email stated that due to the current business climate, difficult decisions had been made, leading to layoffs.
1/1 [==============================] - 0s 72ms/step
How did the layoffs affect the entire team?
The